# Data Science Project Spring 2023

## 200+ Financial Indicators of US stocks (2014-2018)

### Yiwei Gong, Janice Herman, Alexander  Morawietz and Selina Waber

University of Zurich, Spring 2023

## Importing Packages

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_datareader import data


from sklearn.model_selection import train_test_split

## Loading the Data Set


We used the data set from Nicolas Carbone from the webpage [kaggle](https://www.kaggle.com/datasets/cnic92/200-financial-indicators-of-us-stocks-20142018). Each dataset contains over 200 financial indicators, that are found in the [10-K filings](https://www.investopedia.com/terms/1/10-k.asp#:~:text=Key%20Takeaways-,A%2010%2DK%20is%20a%20comprehensive%20report%20filed%20annually%20by,detailed%20than%20the%20annual%20report.) of publicly traded companies from the US between the years 2014 - 2018.

In [ ]:
project_directory = sys.path[0] ## get path of project directory
data_directory = os.path.join(project_directory, 'data')

years = [2014, 2015, 2016, 2017, 2018]

## Loading the yearly dataset into the array dfs
dfs = []
for year in years:
    df = pd.read_csv(os.path.join(data_directory, f'{year}_Financial_Data.csv'), sep=',')
    df['year'] = np.full(df.shape[0], str(year)) ## append column with the year respecitvely
    df['PRICE VAR [%]'] = df[f'{year +1} PRICE VAR [%]'] ## Adding variable of the same name for all df, e.g. '2016 PRICE VAR [%]' renamed to 'PRICE VAR [%]'
    df = df.drop(columns=[f'{year +1} PRICE VAR [%]']) # dropp year-specific variable name
    df.columns.values[0] = 'Stock Name' # name the first variable 
    dfs.append(df)
    
    
df = pd.concat(dfs, ignore_index=True) ## concat the diffrent dataframes
df.head()

## Some Explanation of Variables:

### Adding  `year` as a cathegorical variable

We added a column named `year` which contains the respecitve year.



### Handling the variable `Price VAR [%]`

The last column, `PRICE VAR [%]`, lists the percent price variation of each stock for the year. For example, if we consider the dataset 2015_Financial_Data.csv, we will have:

- 200+ financial indicators for the year 2015;
- percent price variation for the year 2016 (meaning from the first trading day on Jan 2016 to the last trading day on Dec 2016).

We renamed all the variables with the specific year in it, e.g.  `2016 PRICE VAR [%]` to `PRICE VAR [%]`. We dropped the old ones.Now we just have one variable `PRICE VAR [%]`. 


### The variable `class`

class lists a binary classification for each stock, where

- for each stock, if the PRICE VAR [%] value is positive, class = 1. From a trading perspective, the 1 identifies those stocks that an hypothetical trader should BUY at the start of the year and sell at the end of the year for a profit.
- for each stock, if the PRICE VAR [%] value is negative, class = 0. From a trading perspective, the 0 identifies those stocks that an hypothetical trader should NOT BUY, since their value will decrease, meaning a loss of capital.


The columns `PRICE VAR [%]` and `class` make possible to use the datasets for both classification and regression tasks:

- If the user wishes to train a machine learning model so that it learns to classify those stocks that in buy-worthy and not buy-worthy, it is possible to get the targets from the class column;
- If the user wishes to train a machine learning model so that it learns to predict the future value of a stock, it is possible to get the targets from the PRICE VAR [%] column.


### The variable  `Stock Name`

We named the first variable `Stock Name`since it has not been named in the original dataset.



## First Description of the Data

In [ ]:
df.info(verbose=True)

## Numerical and Catgorical Features/Variables

We are converting `Class`to a cathegorical variable.

In [ ]:
df['Class'] = df.Class.astype('object') ## object or catheogry?? whats the difference???

numCols = list(df.select_dtypes(exclude='object').columns)
print(f"There are {len(numCols)} numerical features:\n")

catCols = list(df.select_dtypes(include='object').columns)
print(f"There are {len(catCols)} categorical features:\n", catCols)

## Any Duplicates? 

No, there are no duplicates for rows but there are 20 duplicates for columns/ 10 each. Not same variable name but same data!

In [ ]:
print(f'Duplicates in Rows:', True in list(df.duplicated()))


In [ ]:
print(f'Duplicates in Columns:', True in list(df.T.duplicated().T))
print("Show the Duplicates:")
print(df.T[df.T.duplicated(keep=False)].T)

Our Duplicates are the following pairs:

- `ebitperRevenue` and `eBITperRevenu`
- `ebtperEBIT` and `eBTperEBIT`
- `niperEBT` and `nIperEBT`
- `returnOnAssets` and `Return on Tangible Assets`
- `returnOnCapitalEmployed` and `ROIC`
- `payablesTurnover` and `Payables Turnover`
- `inventoryTurnover` and `Inventory Turnover`
- `debtRatio` and `Debt to Assets`
- `debtEquityRatio` and `Debt to Equity`
- `cashFlowToDebtRatio` and `cashFlowCoverageRatios`

We will remove the first occurence of the duplicates respectively.

In [ ]:
shape_old=df.shape

## HUGHE PROBLEM --> df.drop_duplicates() transforms all variables to the datatype 'object'!
#I don't know why it does that!
# So i will remove the manually!!!

#df= df.T.drop_duplicates().T # remove duplicates!

df= df.drop(columns=['eBITperRevenue', 'eBTperEBIT', 'nIperEBT', 'Return on Tangible Assets', 
                     'ROIC', 'Payables Turnover', 'Inventory Turnover', 'Debt to Assets', 'Debt to Equity', 
                     'cashFlowCoverageRatios'])

print(f' Shape with duplicates:', shape_old) 
print(f' Shape after removal of duplicates:', df.shape) 


#print(df.info(verbose=True)) ok good, sill folat64 objects


## Class Balance?

The Variable `Class`is not balanced. We have to keep that in mind for train and test split. 

In [ ]:
# Distribution of target Variable
#sns.barplot(x = '0', y = '1', data = df['Class'].value_counts())
#plt.show()

#That won't work??? WHY????

## Variation 

There is something seriously wrong with the variable `operatingProfitMargin`. It has a Standard Deviation of 0, which cant be! So we dropp it.

In [ ]:
df.std(axis=0, skipna=True, numeric_only=True)

no_std= [ std for std in df.std( axis=0, skipna=True, numeric_only=True) if std <= 0.15]
print(no_std)



#x = df.std( axis = 0, skipna = True, numeric_only = True)
#[(index, x[index]) for index in x.index.values if x[index] > 0]

### TO DOOO!!!!!!!!!!!!!!!!!

df= df.drop(columns= ['operatingProfitMargin'])


## Outliers Dedection for `PRICE VAR[%]`

In [ ]:
df_ = df.loc[:, ['Sector','PRICE VAR [%]']]

# Get list of sectors
sector_list = df_['Sector'].unique()

# Plot the percent price variation for each sector
for sector in sector_list:
    
    temp = df_[df_['Sector'] == sector]

    plt.figure(figsize=(30,5))
    plt.plot(temp['PRICE VAR [%]'])
    plt.title(sector.upper())
    plt.show()
    

### Outliers

 copy paste from here https://www.kaggle.com/code/cnic92/explore-and-clean-financial-indicators-dataset

In [ ]:
# Get stocks that increased more than 500%
gain = 500
top_gainers = df[df['PRICE VAR [%]'] >= gain]
top_gainers = top_gainers['PRICE VAR [%]'].sort_values(ascending=False)
print(f'{len(top_gainers)} STOCKS with more than {gain}% gain.')
print()



In [ ]:
# Would be cool if that would work.........

# Set
date_start = '01-01-2015'
date_end = '12-31-2015'
tickers = top_gainers.index.values.tolist()

for ticker in tickers:
    
    # Pull daily prices for each ticker from Yahoo Finance
    daily_price = data.DataReader(ticker, 'yahoo', date_start, date_end)
    
    # Plot prices with volume
    fig, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]})
    
    ax0.plot(daily_price['Adj Close'])
    ax0.set_title(ticker, fontsize=18)
    ax0.set_ylabel('Daily Adj Close $', fontsize=14)
    ax1.plot(daily_price['Volume'])
    ax1.set_ylabel('Volume', fontsize=14)
    ax1.yaxis.set_major_formatter(
            matplotlib.ticker.StrMethodFormatter('{x:.0E}'))

    fig.align_ylabels(ax1)
    fig.tight_layout()
    plt.show()

## Outliers cleaning

There are outliers/extreme values that are probably caused by mistypings. During our analysis of the data, we noticed that the values of NA and 0 were frequently used. We realized that 0 was used interchangeably with NA.  Also there are a lot of values that seem impossible. 

In [ ]:
# IQR
test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
for col in test:
    iqr= stats.iqr(df[col], nan_policy='omit')
    print(iqr)

#''''''


https://www.kaggle.com/code/nareshbhat/outlier-the-silent-killer

## Missing Values

There are a lot of missing values. 

In [ ]:
print(f'There are in total {df.isnull().sum().sum()} NAN in the dataframe')

In [ ]:
#df.replace([np.inf, -np.inf], np.nan, inplace=True) do that in Outliers Section!
#df.replace(to_replace = 0, value = np.nan, inplace=true) ????? DA EVTL noch mehr mit NAN ersetzen!

In [ ]:
## Overview of all variables with missing values
df.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(100,20))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Quantifying ALL missing data')


In [ ]:
most_nan = df.isnull().mean().sort_values(ascending=False)
most_nan = most_nan[most_nan > 0.3]

most_nan.plot.bar(figsize=(20,20))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Data with more than 30% missing')


In [ ]:
# Percentage of missing values for the variables
missing = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Percent of Missing Values'])
missing_data.head(25)

In [ ]:
# Plot missing values 2.0
sns.heatmap(df.isna().transpose(), cmap="Blues", cbar_kws={'label': 'Missing Values'});

## Assigning X and target y

In [ ]:
X = df.loc[:, df.columns != "Class"]
y = df["Class"] 


## Adding Dummies

In [ ]:
# Factorize categorical values, assign output to X
# create (multiple) dummy variables for a categorical variable




## Train and Test Split

In [ ]:
## Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42) 

## Important Link

[Cleaning Data Set](https://www.kaggle.com/code/cnic92/explore-and-clean-financial-indicators-dataset)

## Neural Network

In [ ]:
import torch
from torchvision import datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision

from skorch import NeuralNetClassifier

from sklearn.model_selection import GridSearchCV

In [ ]:
X = df.drop(labels=["Class", "Stock Name", "Sector", "year", "PRICE VAR [%]"], axis = 1)
y = df["Class"]

### Tentative procedure for NaN values

In [ ]:
#Drop all columns with nr. of NaN > 30%

print("Shape of X data:", X.shape)

most_nan_columns = X.isnull().mean(axis = 0)
most_nan_columns = most_nan_columns[most_nan_columns > 0.3]

X = X.drop(columns=most_nan_columns.index.values)

print("Shape of X data after removing columns with nr. of NaN > 30%:", X.shape)

In [ ]:
#Drop all rows with nr. of NaN > 30%

most_nan_rows = X.isnull().mean(axis = 1)
most_nan_rows = most_nan_rows[most_nan_rows > 0.3]

X = X.drop(labels=most_nan_rows.index.values, axis = 0)
y = y.drop(labels=most_nan_rows.index.values, axis = 0)

print("Shape of X, y data after removing rows with nr. of NaN > 30%:", X.shape, y.shape)

In [ ]:
#Replace all remaining NaN with mean value (this technique is explicitely not recommended)

X = X.fillna(X.mean())

In [ ]:
#Normalization of the data

X = (X-X.mean())/X.std()

### FCNetwork

In [ ]:
class FCNetwork(nn.Module):
    def __init__(self, dropout_rate=0.3, input_dim = 192):
        super().__init__()
        self.lin1 = nn.Linear(input_dim, input_dim//2)
        self.lin2 = nn.Linear(input_dim//2, input_dim//4)
        self.lin3 = nn.Linear(input_dim//4, 1)
        self.drop1 = nn.Dropout(p = dropout_rate)
        self.drop2 = nn.Dropout(p = dropout_rate)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.drop1(x)
        x = F.relu(self.lin2(x))
        x = self.drop2(x)
        x = self.lin3(x)
        x = self.prob(x)
        return x

In [ ]:
batch_size = 4

In [ ]:
input_dim = X.shape[1]

### Grid Search for optimal Hyperparameters

In [ ]:
#Transformation of the data to tensors for the Grid Search
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.astype("int32").values, dtype=torch.float32).reshape(-1, 1)

In [ ]:
param_grid = {"optimizer__lr": [0.001, 0.005, 0.01], 
              "optimizer__momentum": [0.6, 0.8, 0.9],
              "module__dropout_rate": [0.2, 0.3, 0.4]}

In [ ]:
model = NeuralNetClassifier(
    FCNetwork,
    criterion=nn.BCELoss,
    optimizer=optim.SGD,
    max_epochs=10,
    batch_size=batch_size,
    verbose=False
)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs = -1)
grid_result = grid.fit(X_tensor, y_tensor)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Training the Neural Network with the optimal parameters

In [ ]:
optimal_lr = grid_result.best_params_["optimizer__lr"]
optimal_momentum = grid_result.best_params_["optimizer__momentum"]
optimal_dropout = grid_result.best_params_["module__dropout_rate"]

In [ ]:
#Definition of Dataset for Dataloader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.astype("int32").values, dtype=torch.float32).reshape(-1, 1)
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
#Creation of train-, evaluation- and test-set
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.2, random_state = 42)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
#Creation of the Dataloader
trainloader = DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

valloader = DataLoader(val_dataset, batch_size=batch_size,
                                         shuffle=False)

testloader = DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [ ]:
network = FCNetwork(input_dim = input_dim, dropout_rate = optimal_dropout)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.SGD(network.parameters(), lr=optimal_lr, momentum=optimal_momentum)

In [ ]:
PATH = './net.pth'

In [ ]:
min_val_loss = float("inf")
for epoch in range(500):  # loop over the dataset multiple times

    running_loss = 0.0
    network.train()
    n = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        n += 1
    
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / n:.3f}')

    network.eval()
    val_loss = 0
    n = 0
    with torch.no_grad():
        for i, data in enumerate(valloader, 0):
            inputs, labels = data

            outputs = network(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            n += 1
            
            
            #if i % 2000 == 1999:    # print every 2000 mini-batches
                #print(f'[{epoch + 1}, {i + 1:5d}] loss: {val_loss / 2000:.3f}')
                #val_loss = 0.0
    
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {val_loss / n:.3f}')            
                
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        print(f"The new best model is at epoch {epoch}")
        torch.save(network.state_dict(), PATH)
    print(f'Epoch: {epoch} over')

In [ ]:
network = FCNetwork()
network.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
network.eval()
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        # calculate outputs by running images through the network
        outputs = network(inputs)
        # the class with the highest energy is what we choose as prediction
        predicted = outputs>0.5 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total} %')